1. Packages 설치
- `%%capture`
> Jupyter Notebook에서 셀의 출력을 숨기는 옵션 (긴 설치 과정 로그 감추기)

- Unsloth
> LLAMA 모델의 fine-tuning을 효율적으로 쉽게 만들어 주는 도구
> `[colab-new]`는 Google Colab 환경에 맞는 버전을 설치하는 옵션

```cmd
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
```

- fine-tuning 필요한 추가 라이버리
> `--no-deps` 옵션은 라이브러리들의 종속성을 설치하지 않도록 설정

```cmd
!pip install --no-deps xformers trl peft accelerate bitsandbytes
```

- 종속성 라이브러리
> `xformers` : 트랜스포머 모델의 성능을 향상시키는 라이브러리

> `trl` : 강화학습을 이용한 언어 모델 훈련을 위한 라이브러리

> `peft` : 매개변수 효율적 미세 조정(Parameter-Efficient Fine-Tuning)을 위한 라이브러리

> `accelerate` : 딥러닝 모델의 훈련을 가속화 하는 라이브러리

> `bitsandbytes` : 모델 양자화를 위한 라이브러리

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# import torch
major_version = int(torch.__version__.split('.')[0])
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

2. Unsloth 실행
- 모델이 처리 할 수 있는 최대 시퀀스 길이 설정
> `RoPE(Rotary Position Embedding)` 스케일링을 자동으로 지원하므로 원하는 값으로 설정 가능

```python
max_seq_length = 2048
```

- 모델의 데이터 타입을 설정 `None`으로 두면 자동으로 감지
> `Tesla T4, V100 GPU`의 경우 `Float16, Ampere` 이상의 GPU에서 `Bfloat16`을 사용

```python
dtype = None
```

- 4비트 양자화를 사용할지 설정, 메모리 사용량을 줄이는 데 필요
> `True`로 설정하면 4비트 양자화를 사용

> `False`로 설정하면 사용하지 않음

```python
load_in_4bit = True
```

- 미리 4비트로 양자화된 모델들의 목록
> 나열한 모델들은 다운로드가 4배 빠르고 메모리 부족 문제(`OOM`)를 방지

```python
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
]
```

- `FastLanguageModel`
> 사전 훈련된 모델과 토크나이저를 로드

```python
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
```

> PEFT(부분 파인튜닝) 모델을 생성

```python
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # LoRA(Low-Rank Adaptation)의 랭크 값. 작을수록 파라미터 수가 적음.
    lora_alpha = 32, # LoRA scaling factor. 모델이 학습하는 속도나 성능에 영향을 줄 수 있음.
    lora_dropout = 0.05, # LoRA 학습 시 적용할 드롭아웃 비율. 과적합 방지를 위함.
    # LoRA를 적용할 모델 내 모듈들 (주로 attention 및 feed-forward network의 projection layer들)
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj",],
    bias = "none", # bias 항을 학습하지 않음. ("none", "all", "lora_only" 가능)
    use_gradient_checkpointing = "unsloth", # gradient checkpointing을 통해 메모리 절약. "unsloth"는 특정 최적화 방식 사용.
    random_state = 123, # 결과 재현성을 위한 랜덤 시드 설정
    use_rslora = False, # rslora (rank-stable LoRA) 사용 여부. False면 일반 LoRA 사용
    loftq_config = None, # LoFTQ (Low-rank + Quantization) 관련 설정. None이면 미사용
)
```

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True
model_name = ""

fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0.05,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 123,
    use_rslora = False,
    loftq_config = None,
)

3. 데이터셋 준비하기
- HuggingFace 또는 Local (`jsonl`) 사용하기
```python
from datasets import load_dataset
data_model = ""
dataset = load_dataset(data_model, split="train")
```

from datasets import load_dataset
data_model = ""
dataset = load_dataset(data_model, split="train")

4. 주어진 예시들을 포맷팅하는 함수 및 데이터셋에 formatting_prompts_func 함수
- `Tokenizer`의 EOS(`End of Sequence`) 토큰을 가져오기
```python
EOS_TOKEN = tokenizer.eos_token
```

- `Alpaca` 형식의 프롬프트 템플릿 정의
```python
alpaca_prompt = """

### Instruction:
{0}

### Input:
{1}

### Response:
{2}"""
```

In [ ]:
EOS_TOKEN = tokenizer.eos_token

alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["QUESTION"]
    outputs = examples["ANSWER"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

5. Huggingface TRL의 SFTTrainer를 사용하여 모델 훈련하기

from trl import SFTTrainer, SFTConfig

tokenizer.padding_side = "right"

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    eval_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    ),
)

6. 모델을 훈련시키고 통계를 반환

In [ ]:
trainer_stats = trainer.train()

7. 모델 실행하기 (추론)

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )

stop_token = "<|end_of_text|>"
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]

stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)

8 테스트 예시

In [ ]:
from transformers import TextStreamer

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "GPT-4와 GPT-3.5 터보는 'LLM 환각 지수' 평가에서 어떤 성능을 보였습니까?",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=2048,
    stopping_criteria=stopping_criteria
)